### Import mix module 

In [9]:
import numpy as np
import pandas as pd
from CODECbreakCode.AudioMixer import SingleFileAudioMixer
import CODECbreakCode.Evaluator as Evaluator
from CODECbreakCode.Evaluator import MeasureHAAQIOutput
import CODECbreakCode.NoiseEval as NEUtil
import argparse

In [10]:
import warnings
warnings.filterwarnings("ignore", "Possible clipped samples in output.")
warnings.filterwarnings("ignore",message="Warning: input samples dtype is np.float64. Converting to np.float32")

In [11]:
Mixing_Path = '/home/codecrack/CodecBreakerwithRL/AudioEX/'
Noise_Generator_MP3 = SingleFileAudioMixer(filename="reggae.wav", foldpath=Mixing_Path, TrackType = NEUtil.MixingType.File)
Referece_File,_ = Noise_Generator_MP3.TestNoisedOnlyFile([0,0,0,0],"Reference.wav")
Referece_File
Referece_MP3File = Evaluator.Mp3LameLossyCompress(Referece_File,64)
print(f"Referece_MP3File:{Referece_MP3File}")

Audio duration orginal is 17.462666666666667 seconds, now is the 8.0, the audio changing to the MONO
After LUFS, the mixing ouput in the RMS, Total: -9.11dB, Clipping Ratio&Cliped Num: (0.002002604166666667, 769)
Referece_MP3File:/home/codecrack/CodecBreakerwithRL/AudioEX/Mixing_Result_Mp3_Wav/Reference_64kbps.wav


In [12]:
####initalise the Haaqi
MeasureHAAQI = MeasureHAAQIOutput(Referece_MP3File)#Initilize the HAAQI with a permanent reference
MeasureHAAQI.MeasureHAQQIOutput(Referece_MP3File) #Test on how far from itself to itself

0.9985486851271597

In [13]:
def haaqi_reward_fn(solution: np.ndarray) -> float:
    thre = solution[0]
    thre = thre - 15
    ratio = solution[1]
    ratio = ratio + 4
    attack = solution[2]
    attack = attack + 10.1
    release = solution[3] 
    release = release + 300.1
    degradated_filename = f'audio_mixing_reggea_dynC_thre_{thre}_ratio_{ratio}_attk_{attack}_rele_{release}.wav'
    gener_Audio = Noise_Generator_MP3.TestNoisedOnlyFileOnlyDynamicCompressor(outputfilename = degradated_filename,
                                                               threshold = thre,
                                                               ratio = ratio,
                                                               attack = attack,
                                                               release = release)
    gener_Audio_mp3 = Evaluator.Mp3LameLossyCompress(gener_Audio,64)
    score = MeasureHAAQI.MeasureHAQQIOutput(gener_Audio_mp3)
    print(f"Current degraded thre:{thre}, ratio: {ratio}, attack:{attack},release:{release} bring the HAAQI score to {score}")
    return 1-score

In [14]:
haaqi_reward_fn(np.array([1, 0, 1, -50]))

After LUFS, the mixing ouput in the RMS, Total: -9.12dB, Clipping Ratio&Cliped Num: (0.0019036458333333334, 731)
Current degraded thre:-14, ratio: 4, attack:11.1,release:250.10000000000002 bring the HAAQI score to 0.9027331951766844


0.09726680482331562

### Import Discrete RL module

In [ ]:
import Optimiser

In [ ]:
import Optimiser.discrete_RL_train
from Optimiser.discrete_RL_train import discrete_RL_train as RLTrain

In [ ]:
trainner = RLTrain()
trainner.set_environments(haaqi_reward_fn)
trainner.train()

In [26]:
haaqi_reward_fn(np.array([ -3.9 ,  -0.5 ,  -4.4 , -52.72]))

After LUFS, the mixing ouput in the RMS, Total: -9.24dB, Clipping Ratio&Cliped Num: (0.0011848958333333334, 455)
Current degraded thre:-18.9, ratio: 3.5, attack:5.699999999999999,release:247.38000000000002 bring the HAAQI score to 0.8496384255903743


0.15036157440962572

In [28]:
Noise_Generator_MP3.EraseTheMp3Mixing()

In [29]:
haaqi_reward_fn(np.array([-12.4, 2.7, -6.8, -56]))

After LUFS, the mixing ouput in the RMS, Total: -9.47dB, Clipping Ratio&Cliped Num: (0.00075, 288)
Current degraded thre:-27.4, ratio: 6.7, attack:3.3,release:244.10000000000002 bring the HAAQI score to 0.813212518280853


0.18678748171914705

### Import Continous RL

In [30]:
import Optimiser

In [31]:
import Optimiser.continous_RL_train
from Optimiser.continous_RL_train import continous_RL_train as CRLTrain

2025-07-11 17:07:58.527106: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-11 17:07:58.595938: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-11 17:07:58.595964: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-11 17:07:58.597235: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-11 17:07:58.607414: I tensorflow/core/platform/cpu_feature_guar

In [ ]:
trainner = CRLTrain()
trainner.set_environments(haaqi_reward_fn)
trainner.train()

After LUFS, the mixing ouput in the RMS, Total: -9.11dB, Clipping Ratio&Cliped Num: (0.0019401041666666666, 745)
Current degraded thre:-12.613500595092773, ratio: 4.485066801309586, attack:10.939274048805236,release:249.32033691406252 bring the HAAQI score to 0.926677576247475
After LUFS, the mixing ouput in the RMS, Total: -9.11dB, Clipping Ratio&Cliped Num: (0.00196875, 756)
Current degraded thre:-11.659292221069336, ratio: 4.749298393726349, attack:10.671702301502227,release:249.6432929992676 bring the HAAQI score to 0.9450554756101999
After LUFS, the mixing ouput in the RMS, Total: -9.12dB, Clipping Ratio&Cliped Num: (0.0019010416666666668, 730)
Current degraded thre:-13.499142050743103, ratio: 4.718492865562439, attack:10.099718070030212,release:250.71190032958987 bring the HAAQI score to 0.908530365969612
After LUFS, the mixing ouput in the RMS, Total: -9.11dB, Clipping Ratio&Cliped Num: (0.001966145833333333, 755)
Current degraded thre:-11.927690505981445, ratio: 4.0416597127914

### Import GA module

In [21]:
import Optimiser.genetic_evo
from Optimiser.genetic_evo import GeneticOptimiser

In [22]:
GA_Opt = GeneticOptimiser()

In [23]:
GA_Opt.ga_init_env()

In [24]:
GA_Opt.set_fitnessfun(haaqi_reward_fn)

In [25]:
GA_Opt.run(num_generations = 5,sol_per_pop = 6)

sol_per_pop: 6, num_genes: 4, num_generations: 5, preset_population: [[1.0, 0.0, 1.0, -50.0]]
After LUFS, the mixing ouput in the RMS, Total: -9.26dB, Clipping Ratio&Cliped Num: (0.0016640625, 639)


/root/anaconda3/envs/codecbreak/lib/python3.10/site-packages/pygad/pygad.py:753: UserWarning: The percentage of genes to mutate (mutation_percent_genes=0.2) resulted in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(f"The percentage of genes to mutate (mutation_percent_genes={mutation_percent_genes}) resulted in selecting ({mutation_num_genes}) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).\nIf you do not want to mutate any gene, please set mutation_type=None.")
/root/anaconda3/envs/codecbreak/lib/python3.10/site-packages/pygad/pygad.py:1149: UserWarning: Use the 'save_best_solutions' parameter with caution as it may cause memory overflow when either the number of generations or number of genes is large.
  warnings.warn("Use the 'save_best_solutions' parameter with caution as it may cause memory overflow when either the number of genera

Current degraded thre:-24.3, ratio: 2.9, attack:18.799999999999997,release:147.10000000000002 bring the HAAQI score to 0.8592554057857791
After LUFS, the mixing ouput in the RMS, Total: -9.36dB, Clipping Ratio&Cliped Num: (0.0006328125, 243)
Current degraded thre:-27.4, ratio: 2.3, attack:3.3,release:244.10000000000002 bring the HAAQI score to 0.839448691587177
After LUFS, the mixing ouput in the RMS, Total: -9.11dB, Clipping Ratio&Cliped Num: (0.002002604166666667, 769)
Current degraded thre:-8.6, ratio: 7.9, attack:2.3999999999999995,release:116.10000000000002 bring the HAAQI score to 0.9985486851353156
After LUFS, the mixing ouput in the RMS, Total: -9.11dB, Clipping Ratio&Cliped Num: (0.002002604166666667, 769)
Current degraded thre:-8.5, ratio: 3.7, attack:6.5,release:521.1 bring the HAAQI score to 0.9985486851125562
After LUFS, the mixing ouput in the RMS, Total: -9.17dB, Clipping Ratio&Cliped Num: (0.00103125, 396)
Current degraded thre:-17.8, ratio: 2.6, attack:6.5,release:6.10

In [27]:
haaqi_reward_fn(np.array([-12.4, 2.7, -6.8, -56]))

After LUFS, the mixing ouput in the RMS, Total: -9.47dB, Clipping Ratio&Cliped Num: (0.00075, 288)
Current degraded thre:-27.4, ratio: 6.7, attack:3.3,release:244.10000000000002 bring the HAAQI score to 0.813212517707022


0.18678748229297804

In [ ]:
haaqi_reward_fn(np.array([-14.9, -3.8, 6.1, -294]))

In [5]:
GA_Opt._gene_space

[{'low': -15.0, 'high': 15.0, 'step': 0.1},
 {'low': -4.0, 'high': 4.0, 'step': 0.1},
 {'low': -10.0, 'high': 10.0, 'step': 0.1},
 {'low': -300.0, 'high': 300.0, 'step': 1}]